# Sudoku Puzzle Solver 2

Second attempt at building a Sudoku puzzle solver. This time, have put some of the puzzle logic in a class, along with the same sample puzzles from the previous attempt. The class will take care of checking that the puzzle state is valid, and has some helper methods to replace some of the repeated code we had last time.

In [14]:
import sudoku
from IPython.display import HTML, display, clear_output

I've also moved the example puzzles into the class, we can use those to test our solving algorithms. Same examples as before, as well as some others from around the web. Source code has credits/sources.

In [15]:
display(HTML('<ol><li>' + '</li><li>'.join([x['label'] for x in sudoku.SAMPLE_PUZZLES]) + '</li></ol>'))
display(HTML('</li></ol>'))

p = sudoku.SudokuPuzzle(sudoku.SAMPLE_PUZZLES[0]['puzzle'])

## Displaying the puzzle grid

We'll want to take a look at the state of the puzzle so far. We'll print the matrix and show

1. Solved cells
2. Cells with 2 possibilities



In [16]:
def print_puzzle(puzzle):
    display(HTML(puzzle.as_html()))
        

In [17]:
print_puzzle(p)

8,9,,4,,,,5,6
1,4,,3,5,,,9,
,,,,,,8,,
9,,,,,,2,,
,8,,9,6,5,,4,
,,1,,,,,,5
,,8,,,,,,
,3,,,2,1,,7,8
4,2,,,,6,,1,3


## Testing Functions

We'll define a `test_run` function that will generate a new puzzle grid then run a specific "solver" algorithm. There's also a `test_harness` to run that algorithm over a whole set of sample puzzles, and track how long each solver takes.


In [18]:
def test_run(function, puzzle, puzzle_class):
    p = puzzle_class(puzzle)
    assert(not p.is_solved())
    function(p)
    assert(p.is_solved())

Using *Pandas* and a simple data structure to keep track of test times.

In [19]:
import pandas as pd
pd.set_option('precision', 3)

def init_dataframe(skip_levels=['Pathalogical', 'Diabolical']):
    data = {'label': [], 'level': [], 'starting_clues': []}
    for puz in sudoku.SAMPLE_PUZZLES:
        if puz['level'] not in skip_levels:
            data['label'].append(puz['label'])
            data['level'].append(puz['level'])

            tmp = sudoku.SudokuPuzzle(puz['puzzle'])
            data['starting_clues'].append((sudoku.MAX_CELL_VALUE ** 2) - len(tmp.get_all_empty_cells()))
    return data

This `test_harness` will run over all the sample tests in `sudoku.py` (except the ones with a level mentioned in `skip_levels`).

In [20]:
import timeit

NUM_TEST_SAMPLES=3

def test_harness(function, data, skip_levels=['Pathalogical', 'Diabolical'], puzzle_class=sudoku.SudokuPuzzle):
    """
    Given a `function` to solve a puzzle, and a `data` structure to store the timing results, test how long
    the function takes to solve the puzzle. By default will skip "Pathalogical" puzzles.
    """
    test_case_label = f"{function.__name__} ({puzzle_class.__name__})"
    data[test_case_label] = []
    num_puzzles = 0
    total_time = 0
    for puz in sudoku.SAMPLE_PUZZLES:
        if puz['level'] not in skip_levels:
            clear_output(wait=True)
            display(HTML(f"<p>Testing {puz['label']} ({puz['level']})...</p>"))
            t = timeit.timeit('test_run(solver, test_puzzle, puzzle_class)', number=NUM_TEST_SAMPLES, 
                              globals={'test_run': test_run, 'test_puzzle': puz['puzzle'], 'solver': function, 'puzzle_class': puzzle_class})
            num_puzzles += 1
            total_time += t
            data[test_case_label].append(t / NUM_TEST_SAMPLES)
        else:
            print(f"Skipping {puz['label']} ({puz['level']})")
    clear_output()
    display(HTML(f"<p>Tested {num_puzzles} puzzles {NUM_TEST_SAMPLES} times each in {total_time:.2f} seconds.</p>"))

# Stratgegy 1: Brute force (Backtracking)

The previous attempt used deductive reasoning, but that really only got us so far. The `moderate` and harder puzzles remained unsolved. 

One obvious option that we didn't try was to "brute force" the solution using Backtracking. Reasonably good [explanation and visualisation of the process on Wikipedia](https://en.wikipedia.org/wiki/Backtracking#Examples).


In [21]:
def solve_using_backtracking(puzzle):
    """
    Attempts to solve `puzzle` using backtracking. Returns True if puzzle is solved, False if the current solution
    path is a dead-end (results in invalid puzzle). Calls itself recursively.
    """
    empty_cell = puzzle.get_first_empty_cell()
    if len(empty_cell) == 0:
        return True
    
    x, y = empty_cell[0], empty_cell[1]
    for val in range(sudoku.MIN_CELL_VALUE, sudoku.MAX_CELL_VALUE+1):
        if puzzle.is_legal(x, y, val):
            puzzle.set(x, y, val)           
            if solve_using_backtracking(puzzle):
                return True
            else:
                puzzle.clear(x, y)
    return False

In [22]:
p = sudoku.SudokuPuzzle(sudoku.SAMPLE_PUZZLES[0]['puzzle'])
solve_using_backtracking(p)
print_puzzle(p)
assert(p.is_solved())

8,9,3,4,7,2,1,5,6
1,4,6,3,5,8,7,9,2
2,7,5,6,1,9,8,3,4
9,5,4,1,8,3,2,6,7
7,8,2,9,6,5,3,4,1
3,6,1,2,4,7,9,8,5
5,1,8,7,3,4,6,2,9
6,3,9,5,2,1,4,7,8
4,2,7,8,9,6,5,1,3


## Initial evaluation

For the easy puzzles this approach seems to work fine. We could probably speed it up by not bothering with any values we *know* can't be tried because they already exist in that row, column, or cage. But it's pretty fast already for that first (easy) case. Curious to see how long it takes to solve the different puzzle difficulty levels.


In [23]:
data = init_dataframe()
test_harness(solve_using_backtracking, data=data)
df = pd.DataFrame(data)
df

,label,level,starting_clues,solve_using_backtracking (SudokuPuzzle)
0,SMH 1,Kids,31,0.005
1,SMH 2,Easy,24,0.215
2,KTH 1,Easy,30,0.011
3,Rico Alan Heart,Easy,22,0.254
4,SMH 3,Moderate,26,0.155
5,SMH 4,Hard,22,2.682
6,SMH 5,Hard,25,1.086
7,Greg [2017],Hard,21,0.831


The variability in the times comes from the backtracking approach. Because it iterates sequentially towards the possible solution the length of time it takes can vary substantially. There's a test case (level "Pathalogical" called `Rico Alan 3`) with a starting row of "987654321", virtually guaranteeing that the backtracker will have to cycle through and discard an enormous number of potential solutions before finally finding the one that works.

## Variant 1.1: Slightly smarter brute force

So, the first 3 difficulty levels are pretty quick this way, but the last two take ~ 2-5 seconds. Not too bad, but can we make it faster by only trying values that we know are legal?

If we modify the previous backtracking algorithm to not bother looping through all the digits 1..9, but instead use digits that we know *could* be legal. 


In [24]:
def solve_using_backtracking_smarter(puzzle):
    """
    Attempts to solve `puzzle` using backtracking, except this time we only try legal values. Returns True if puzzle is solved, 
    False if the current solution path is a dead-end (results in invalid puzzle). Calls itself recursively.
    """
    empty_cell = puzzle.get_first_empty_cell()
    if len(empty_cell) == 0:
        return True
    
    x, y = empty_cell[0], empty_cell[1]
    for val in puzzle.get_possible_values(x,y):
        puzzle.set(x, y, val)
        if solve_using_backtracking_smarter(puzzle):
            return True
        else:
            puzzle.clear(x, y)
    return False

In [25]:
test_harness(solve_using_backtracking_smarter, data=data)

In [26]:
df = pd.DataFrame(data)
df

,label,level,starting_clues,solve_using_backtracking (SudokuPuzzle),solve_using_backtracking_smarter (SudokuPuzzle)
0,SMH 1,Kids,31,0.005,0.003
1,SMH 2,Easy,24,0.215,0.266
2,KTH 1,Easy,30,0.011,0.015
3,Rico Alan Heart,Easy,22,0.254,0.088
4,SMH 3,Moderate,26,0.155,0.093
5,SMH 4,Hard,22,2.682,1.658
6,SMH 5,Hard,25,1.086,0.786
7,Greg [2017],Hard,21,0.831,0.741


Marginal improvement for the easy to moderate puzzles. Hardly seems worth it. 

There's one more option to try, and that's to select empty cells in a different order, instead of always choosing the *first* one (that search always starts from the "top left", position `(0,0)`.

## Variant 1.2: Backtracking in reverse order

So we'll see if there's any variation when we select an empty cell from the end of the list. I did try a version that selected cells at random, but it seemed to take a very long time to even do the "kids" puzzle. 


In [27]:
import random
import time

def solve_using_backtracking_reverse(puzzle, n=1):
    """
    Attempts to solve `puzzle` using backtracking, trying only legal values, but this time moving "back" through the list of
    empty cells. Returns True if puzzle is solved, False if the current solution path is a dead-end (results in invalid puzzle). 
    Calls itself recursively.
    """
    all_empty_cells = puzzle.get_all_empty_cells()
    if len(all_empty_cells) == 0:
        return True
    empty_cell = all_empty_cells.pop()  # random.choice(all_empty_cells)
    
    x, y = empty_cell[0], empty_cell[1]
    for val in puzzle.get_possible_values(x,y):
        puzzle.set(x, y, val)
        if solve_using_backtracking_reverse(puzzle, n=n+1):
            return True
        else:
            puzzle.clear(x, y)
    return False

In [28]:
test_harness(solve_using_backtracking_reverse, data=data)

In [29]:
df = pd.DataFrame(data)
df

,label,level,starting_clues,solve_using_backtracking (SudokuPuzzle),solve_using_backtracking_smarter (SudokuPuzzle),solve_using_backtracking_reverse (SudokuPuzzle)
0,SMH 1,Kids,31,0.005,0.003,0.044
1,SMH 2,Easy,24,0.215,0.266,0.603
2,KTH 1,Easy,30,0.011,0.015,0.086
3,Rico Alan Heart,Easy,22,0.254,0.088,84.014
4,SMH 3,Moderate,26,0.155,0.093,2.292
5,SMH 4,Hard,22,2.682,1.658,20.925
6,SMH 5,Hard,25,1.086,0.786,0.821
7,Greg [2017],Hard,21,0.831,0.741,0.404


The results for the `Rico Alan Heart` puzzle shows how backtracking can be really sensitive to the actual puzzle itself. In the standard algorithm, most of the first row is already filled in, which cuts the number of possibilities it needs to work through. But when the empty cell ordering is reversed, then the "first" row it processes is mostly empty, so it spends a lot more time on useless combinations.

## Testing with "harder" puzzles

Curious as to how the backtracking functions perform with the so called "pathalogical" test cases.

**Warning:** Don't run this unless you have the time (about an hour).


In [30]:
skip_levels=[] # ['Pathalogical']
pdata = init_dataframe(skip_levels=skip_levels)

In [31]:
test_harness(solve_using_backtracking, data=pdata, skip_levels=skip_levels)
test_harness(solve_using_backtracking_smarter, data=pdata, skip_levels=skip_levels)
test_harness(solve_using_backtracking_reverse, data=pdata, skip_levels=skip_levels)

In [32]:
pdf = pd.DataFrame(pdata)
pdf

,label,level,starting_clues,solve_using_backtracking (SudokuPuzzle),solve_using_backtracking_smarter (SudokuPuzzle),solve_using_backtracking_reverse (SudokuPuzzle)
0,SMH 1,Kids,31,0.016,0.003,0.029
1,SMH 2,Easy,24,0.198,0.210,0.535
2,KTH 1,Easy,30,0.012,0.014,0.094
3,Rico Alan Heart,Easy,22,0.287,0.084,76.176
4,SMH 3,Moderate,26,0.149,0.092,1.966
5,SMH 4,Hard,22,3.134,1.599,17.806
6,SMH 5,Hard,25,1.162,0.689,0.594
7,Greg [2017],Hard,21,0.891,0.675,0.303
8,Rico Alan 1,Diabolical,20,8.600,0.932,149.620
9,Rico Alan 2,Diabolical,20,12.619,33.449,80.445


Interesting...

* Two of the "diabolical" puzzles (`SMH 5`, `Rico Alan 1`) are solved much faster with our slightly improved backtracking solution.  
* `Rico Alan 2` always takes longer (around 3X) with the "smarter" back tracking algorithm and I have no idea why.
* `Rico Alan 3` is designed to be particularly hard on backtracking solvers (first line is "987654321"). Takes around 20 minutes for me (an earlier version of `SudokuPuzzle` class took up to 45 minutes). But reversing the backtracking order significantly speeds up the time to solve.
* The [Qassim Hamza](https://www.flickr.com/photos/npcomplete/2304537670/in/photostream/) puzzle apparently "cannot be solved by humans" but is relatively easy for a backtracking solver. 



# Strategy 2: Possibility Matrix

In the first attempt we used a "possibility matrix" to keep track of all the possible values for a cell. If we ever found a cell that had only one possible value left we'd lock that in, recalculate the possible values for the other cells and repeat the process. This solved some of the easier test puzzles but on its own eventually ran out of options for the harder ones.

So one idea is to first try solving the puzzle this way, then if it fails to completely solve the puzzle we use backtracking.


In [33]:
def solve_using_possibilities(puzzle):
    """
    Scan the `puzzle` for any cells with only one possible value remaining, and write that value into the cell.
    This will update the puzzle with new possible values so we can repeat this process until there are no more cells
    with a single possible value remaining. Returns True if the puzzle is solved, False if it is not.
    """
    if puzzle.is_solved():
        return True
    
    num_total_cells_updated = 0
    num_cycles = 0
    num_cells_updated = 1 # not really, but forcing loop to run at least once
    while num_cells_updated > 0:
        num_cells_updated = 0
        num_cycles += 1
        empties = puzzle.get_all_empty_cells()
        for m in empties:
            possibles = puzzle.get_possible_values(m[0], m[1])
            if len(possibles) == 1:
                (v,) = possibles
                puzzle.set(m[0], m[1], v)
                num_cells_updated += 1
            elif len(possibles) == 0:
                raise ValueError(f"Something has gone wrong -- cell {m[0]},{m[1]} has no possible values left")
        num_total_cells_updated += num_cells_updated
        
    print(f"Updated {num_total_cells_updated} cells in {num_cycles} cycles")    
    return puzzle.is_solved()

def solve_using_combination(puzzle, backtracker=solve_using_backtracking_smarter):
    """
    Calls `solve_using_possibilities` on the puzzle, and if that fails to completely sovle the puzzle then calls
    `solve_using_backtracking_smarter`.
    """
    if solve_using_possibilities(puzzle):
        return True
    print("Puzzle not solved, switching to backtracker")
    return backtracker(puzzle)

In [34]:
p = sudoku.SudokuPuzzle(sudoku.SAMPLE_PUZZLES[0]['puzzle'])
solve_using_combination(p)
print_puzzle(p)

Updated 50 cells in 7 cycles


8,9,3,4,7,2,1,5,6
1,4,6,3,5,8,7,9,2
2,7,5,6,1,9,8,3,4
9,5,4,1,8,3,2,6,7
7,8,2,9,6,5,3,4,1
3,6,1,2,4,7,9,8,5
5,1,8,7,3,4,6,2,9
6,3,9,5,2,1,4,7,8
4,2,7,8,9,6,5,1,3


## Evaluating Possibility Matrix

First we'll check if `solve_using_possibility_matrix` can actually solve all the puzzles on its own.


In [35]:
solvable_with_possibilities = []
not_solvable_with_possibilities = []
for puz in sudoku.SAMPLE_PUZZLES:
    p = sudoku.SudokuPuzzle(puz['puzzle'])
    if solve_using_possibilities(p):
        solvable_with_possibilities.append(puz['label'])
    else:
        not_solvable_with_possibilities.append(puz['label'])
clear_output(wait=True)
display(HTML('<p>Solvable using possibility matrix alone: <ol><li>{}</li></ol></p>'.format('</li><li>'.join(str(_) for _ in solvable_with_possibilities))))
display(HTML('<p>NOT solvable using possibility matrix alone: <ol><li>{}</li></ol></p>'.format('</li><li>'.join(str(_) for _ in not_solvable_with_possibilities))))

Next, we'll repeat the test with all puzzles using the "combination" strategy.

In [36]:
test_harness(solve_using_combination, data=data)

In [37]:
df = pd.DataFrame(data)
df

,label,level,starting_clues,solve_using_backtracking (SudokuPuzzle),solve_using_backtracking_smarter (SudokuPuzzle),solve_using_backtracking_reverse (SudokuPuzzle),solve_using_combination (SudokuPuzzle)
0,SMH 1,Kids,31,0.005,0.003,0.044,0.003
1,SMH 2,Easy,24,0.215,0.266,0.603,0.003
2,KTH 1,Easy,30,0.011,0.015,0.086,0.003
3,Rico Alan Heart,Easy,22,0.254,0.088,84.014,0.088
4,SMH 3,Moderate,26,0.155,0.093,2.292,0.096
5,SMH 4,Hard,22,2.682,1.658,20.925,0.643
6,SMH 5,Hard,25,1.086,0.786,0.821,0.148
7,Greg [2017],Hard,21,0.831,0.741,0.404,0.696


OK, so that's odd -- `SMH 4` takes \~2.5 seconds to solve using backtracking alone (\~18 seconds if the backtracking order is reversed), but only \~1 second of we have a go with the possibility matrix first. How many cells does it fill in?

In [38]:
p = sudoku.SudokuPuzzle(sudoku.SAMPLE_PUZZLES[5]['puzzle'])
print(f"Empty cells before: {p.num_empty_cells()}")
solve_using_possibilities(p)
print(f"Empty cells after: {p.num_empty_cells()}")

Empty cells before: 59
Updated 1 cells in 2 cycles
Empty cells after: 58


So we can only update 1 cell this way, but that's enough to halve the solution time. That's odd -- not something I was expecting, but maybe it makes sense, because having one extra clue would halve the search space for finding a solution?

Let's see how it works on the harder ones...

In [39]:
skip_levels=[] # ['Pathalogical']
test_harness(solve_using_combination, data=pdata, skip_levels=skip_levels)


In [40]:
pdf = pd.DataFrame(pdata)
pdf

,label,level,starting_clues,solve_using_backtracking (SudokuPuzzle),solve_using_backtracking_smarter (SudokuPuzzle),solve_using_backtracking_reverse (SudokuPuzzle),solve_using_combination (SudokuPuzzle)
0,SMH 1,Kids,31,0.016,0.003,0.029,0.002
1,SMH 2,Easy,24,0.198,0.210,0.535,0.004
2,KTH 1,Easy,30,0.012,0.014,0.094,0.003
3,Rico Alan Heart,Easy,22,0.287,0.084,76.176,0.090
4,SMH 3,Moderate,26,0.149,0.092,1.966,0.091
5,SMH 4,Hard,22,3.134,1.599,17.806,0.637
6,SMH 5,Hard,25,1.162,0.689,0.594,0.150
7,Greg [2017],Hard,21,0.891,0.675,0.303,0.674
8,Rico Alan 1,Diabolical,20,8.600,0.932,149.620,0.924
9,Rico Alan 2,Diabolical,20,12.619,33.449,80.445,33.799


# Strategy 3: Constraint Propogation

We can re-use this idea to implement a "constraint propogation" solver. When the backtracker makes a "guess" for a cell, we eliminate that guess from the row, column and cage. If we detect a cell that now has *no* possibilities left, then the guess was in error and we can abort that search path earlier. We can also lock in values for any cell that drops down to a single possibility.

To do this I created a new class `SudokuPuzzleConstrained` which keeps track of the possible values for each cell. It therefore can do a few things a bit differently:

* It generates and then keeps a "cache" of possible values for each cell, which can be retrieved using `get_possible_values`.
* The method `is_legal` will return `False` if writing a value would result in a cell somewhere having no possible values (because that value is the *only* possible value left for a cell in the same row, column, or cage).
* Every time a cell is `set`, it updates the possible values for all cells in that same row, column, and cage to exclude the newly written value. If `set` discovers a cell that now has *no* possible values, it will raise an exception.
* If a cell is cleared with `clear`, it puts the value that *was* in that cell back into the list of possibe values in that row, column, and cage.
* The method `is_puzzle_valid` will return `False` if it discovers a cell that has *no* possible values left.



In [41]:
p = sudoku.SudokuPuzzleConstrained(sudoku.SAMPLE_PUZZLES[1]['puzzle'])
solve_using_backtracking(p)
print_puzzle(p)
assert(p.is_solved())

7,4,3,8,2,1,5,9,6
5,9,6,4,3,7,8,2,1
2,1,8,5,9,6,4,3,7
1,5,7,3,8,2,9,6,4
8,2,4,7,6,9,3,1,5
6,3,9,1,4,5,2,7,8
4,8,2,6,7,3,1,5,9
9,7,5,2,1,8,6,4,3
3,6,1,9,5,4,7,8,2


## Evaluating Constraint Propogation

Let's take a look how this new class performs on the sample test puzzles.


In [42]:
test_harness(solve_using_backtracking, data=data, puzzle_class=sudoku.SudokuPuzzleConstrained)

In [43]:
df = pd.DataFrame(data)
df

,label,level,starting_clues,solve_using_backtracking (SudokuPuzzle),solve_using_backtracking_smarter (SudokuPuzzle),solve_using_backtracking_reverse (SudokuPuzzle),solve_using_combination (SudokuPuzzle),solve_using_backtracking (SudokuPuzzleConstrained)
0,SMH 1,Kids,31,0.005,0.003,0.044,0.003,0.004
1,SMH 2,Easy,24,0.215,0.266,0.603,0.003,0.123
2,KTH 1,Easy,30,0.011,0.015,0.086,0.003,0.008
3,Rico Alan Heart,Easy,22,0.254,0.088,84.014,0.088,0.197
4,SMH 3,Moderate,26,0.155,0.093,2.292,0.096,0.120
5,SMH 4,Hard,22,2.682,1.658,20.925,0.643,2.136
6,SMH 5,Hard,25,1.086,0.786,0.821,0.148,0.928
7,Greg [2017],Hard,21,0.831,0.741,0.404,0.696,0.709


This is marginally better (an earlier implementation of `SudokuPuzzleConstrained` was quite inefficient in its constraint propagation, and could take significantly longer to solve puzzles). The new method is usually a little faster than using backtracking alone. 

Since the `solve_using_backtracking` isn't bothering to utilise the possibility matrix, we should get even better performance if we use `solve_using_backtracking_smarter`. Let's see...


In [44]:
def solve_using_backtracking_cp(puzzle):
    """
    Attempts to solve `puzzle` using backtracking, trying legal values and testing first. Returns True if puzzle is solved, 
    False if the current solution path is a dead-end (results in invalid puzzle). Calls itself recursively.
    """
    empty_cell = puzzle.get_first_empty_cell()
    if len(empty_cell) == 0:
        return True
    
    x, y = empty_cell[0], empty_cell[1]
    for val in puzzle.get_possible_values(x,y):
        try:
            puzzle.set(x, y, val)
        except ValueError:
            continue
            
        if solve_using_backtracking_cp(puzzle):
            return True
        else:
            puzzle.clear(x, y)
    return False

In [45]:
test_harness(solve_using_backtracking_cp, data=data, puzzle_class=sudoku.SudokuPuzzleConstrained)

In [46]:
df = pd.DataFrame(data)
df

,label,level,starting_clues,solve_using_backtracking (SudokuPuzzle),solve_using_backtracking_smarter (SudokuPuzzle),solve_using_backtracking_reverse (SudokuPuzzle),solve_using_combination (SudokuPuzzle),solve_using_backtracking (SudokuPuzzleConstrained),solve_using_backtracking_cp (SudokuPuzzleConstrained)
0,SMH 1,Kids,31,0.005,0.003,0.044,0.003,0.004,0.002
1,SMH 2,Easy,24,0.215,0.266,0.603,0.003,0.123,0.131
2,KTH 1,Easy,30,0.011,0.015,0.086,0.003,0.008,0.009
3,Rico Alan Heart,Easy,22,0.254,0.088,84.014,0.088,0.197,0.052
4,SMH 3,Moderate,26,0.155,0.093,2.292,0.096,0.120,0.055
5,SMH 4,Hard,22,2.682,1.658,20.925,0.643,2.136,0.976
6,SMH 5,Hard,25,1.086,0.786,0.821,0.148,0.928,0.415
7,Greg [2017],Hard,21,0.831,0.741,0.404,0.696,0.709,0.423


Better.

The approach could be optimised further. After setting a value to a cell, we could check if any cells now had exactly one possible value left, immediately set that, and then repeat the process until we run out of single-value options. Then continue the backtracking. This is really how constraint propagation should work. Will come back to that...

What about the "pathalogical" test examples?


In [47]:
test_harness(solve_using_backtracking_cp, data=pdata, skip_levels=skip_levels)


In [48]:
pdf = pd.DataFrame(pdata)
pdf

,label,level,starting_clues,solve_using_backtracking (SudokuPuzzle),solve_using_backtracking_smarter (SudokuPuzzle),solve_using_backtracking_reverse (SudokuPuzzle),solve_using_combination (SudokuPuzzle),solve_using_backtracking_cp (SudokuPuzzle)
0,SMH 1,Kids,31,0.016,0.003,0.029,0.002,0.004
1,SMH 2,Easy,24,0.198,0.210,0.535,0.004,0.212
2,KTH 1,Easy,30,0.012,0.014,0.094,0.003,0.015
3,Rico Alan Heart,Easy,22,0.287,0.084,76.176,0.090,0.084
4,SMH 3,Moderate,26,0.149,0.092,1.966,0.091,0.091
5,SMH 4,Hard,22,3.134,1.599,17.806,0.637,1.586
6,SMH 5,Hard,25,1.162,0.689,0.594,0.150,0.681
7,Greg [2017],Hard,21,0.891,0.675,0.303,0.674,0.676
8,Rico Alan 1,Diabolical,20,8.600,0.932,149.620,0.924,0.929
9,Rico Alan 2,Diabolical,20,12.619,33.449,80.445,33.799,34.941


Interesting bits:

* The `Rico Alan 2` puzzle completes in \~12 seconds using the naive backtracker, which is the best performance among all the methods. I would have thought the constraint propogation solution could do better than \~34 seconds, and so far I have no idea why it doesn't.
* Other puzzles, such as `Rico Alan Border #1` and `Rico Alan #3` are absolutely terrible with the naive backtracker but can be solved significantly faster using constraint propogation.
* Outside this Notebook I've been using the [Python profiler](https://docs.python.org/3/library/profile.html) to get an idea of where performance issues are. I could bring that analysis into the Notebook for more data practice.
* Part of this exercise is learning Python and Jupyter. The data above could be better represented graphically so will try that next.


# Conclusion

Some progress was made over the [previous attempt](Sudoku+Solver.ipyb):

* All puzzles can be solved with the backtracking approach
* A combination of "possibility matrix" and "backtracking" performs faster than backtracking alone
* A proper constraint propagation approach, when combined with backtracking, should solve any puzzle in the fastest time, but there are clearly examples where the naive approach is fastest. Which is kinda interesting.

So, next steps:

1. Update `SudokuPuzzleConstrained` to find single-value option cells, and update those cells
2. Find some more "pathalogical" test cases that are supposed to be punishing for backtrackers
3. Check out some of the more advanced techniques (e.g. "Solution X" and "dancing links")


# Appendix

## Sources

Part of this exercise was to learn Python and Jupyter skills while also solving a problem that I found interesting. So I've largely avoided reading other people's solutions to solving Sudoku. However from time to time I've gotten stuck or just been curious about something and found the below sources useful.

* [Simple sudoku solver using constraint propagation](https://gpicavet.github.io/jekyll/update/2017/12/16/sudoku-solver.html) (Greg's Blog).
* [Sudoku solving algorithms](https://en.wikipedia.org/wiki/Sudoku_solving_algorithms) -- links to some sample puzzles on Flickr of all places. Found via the [Wikipedia article on Sudoku solving algorithms](https://en.wikipedia.org/wiki/Sudoku_solving_algorithms).
* [AI Sudoku](http://www.aisudoku.com/index_en.html) -- collection of "really hard" puzzles.
* The [sudoku.py](sudoku.py) class has URLs to where I found some of the sample puzzles. I've attempted to use labels for them that credit the source, although it's not always clear where the original puzzle came from.

## Table formatting

Snippet below inserts some CSS to make the table look more like a Sudoku puzzle grid.


In [49]:
display(HTML('''
<style type="text/css">
.sudoku td {
    width: 40px;
    height: 40px;
    border: 1px solid #000 !important; 
    text-align: center !important;
}

.sudoku td:nth-of-type(3n) {    
    border-right: 3px solid red !important;
}

.sudoku tr:nth-of-type(3n) td {    
    border-bottom: 3px solid red !important;
}

.sudoku table {
    border: 3px solid red !important;
}

.sudoku-solved table {
    border: 3px solid green !important;
}
</style>
'''))

